In [ ]:
import numpy as np
import pandas as pd
import time
import os

In [ ]:
def loadNLPVectors(folder, filename):
    file = folder + '/' + filename + '.npy'
    return np.load(file)

In [ ]:
def loadData(name):
    X_train = loadNLPVectors('train', name)
    y_train = loadNLPVectors('train', 'labels')
    
    X_test = loadNLPVectors('test', name)
    y_test = loadNLPVectors('test', 'labels')
    
    train = [X_train, y_train]
    test = [X_test, y_test]
    
    return train, test

# Load NLP Training/Testing Data

In [ ]:
train_uni, test_uni = loadData('unigram')

In [ ]:
train_big, test_big = loadData('bigram')

In [ ]:
train_tfidf, test_tfidf = loadData('tfidf')

In [ ]:
train_word2vec, test_word2vec = loadData('word2vec')

In [ ]:
train_runi, test_runi = loadData('runigram')

In [ ]:
train_rbig, test_rbig = loadData('rbigram')

In [ ]:
train_rtfidf, test_rtfidf = loadData('rtfidf')

# K-Means Clustering

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
def kCluster(train, verbose = 0):
    cluster = KMeans(n_clusters = 2, verbose = verbose)
    start = time.time()
    cluster.fit(train[0], train[1])
    end = time.time() - start
    print('Training Time: ' + str(end))
    return cluster

# Evaluate Clusters

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
def divideTestData(test):
    X = test[0]
    y = test[1]
    
    neutral_array = []
    hate_array = []
    
    for i in range(0, len(y)):
        if y[i] == 0:
            neutral_array.append(X[i])
        elif y[i] == 1:
            hate_array.append(X[i])
    
    neutral = np.asarray(neutral_array)
    hate = np.asarray(hate_array)
    
    return neutral, hate

In [ ]:
def getClusterAvg(cluster_pred):
    total = len(cluster_pred)
    count_0 = 0
    count_1 = 0
    for i in range(0, total):
        if cluster_pred[i] == 0:
            count_0 += 1
        elif cluster_pred[i] == 1:
            count_1 += 1
    
    cluster0 = count_0/total
    cluster1 = count_1/total
    
    return cluster0, cluster1

In [ ]:
def evaluateCluster(cluster, test):
    neutral, hate = divideTestData(test)
    
    neutral_pred = cluster.predict(neutral)
    hate_pred = cluster.predict(hate)
    
    nCluster0, nCluster1 = getClusterAvg(neutral_pred)
    hCluster0, hCluster1 = getClusterAvg(hate_pred)
    
    print("Neutral Cluster = 0: " + str(nCluster0))
    print("Hate Cluster = 1:    " + str(hCluster1))
    print(" ")
    print("Neutral Cluster = 1: " + str(nCluster1))
    print("Hate Cluster = 0:    " + str(hCluster0))
    

# Reduced Unigram Clustering

In [ ]:
k_runi = kCluster(train_runi)

In [ ]:
evaluateCluster(k_runi, test_runi)

# Reduced Bigram Clustering

In [ ]:
k_rbig = kCluster(train_rbig)

In [ ]:
evaluateCluster(k_rbig, test_rbig)

# Reduced TFIDF Clustering

In [ ]:
k_rtfidf = kCluster(train_rtfidf)

In [ ]:
evaluateCluster(k_rtfidf, test_rtfidf)

# Word2Vec Clustering

In [ ]:
k_vec = kCluster(train_vec)

In [ ]:
evaluateCluster(k_vec, test_vec)

# Unigram Clustering

In [ ]:
k_uni = kCluster(train_uni)

In [ ]:
evaluateCluster(k_uni, test_uni)

# Bigram Clustering

In [ ]:
k_big = kCluster(train_big)

In [ ]:
evaluateCluster(k_big, test_big)

# TFIDF Clustering

In [ ]:
k_tfidf = kCluster(train_tfidf)

In [ ]:
evaluateCluster(k_tfidf, test_tfidf)